In [14]:
__imp

Libraries have been loaded


## Load Additional Libraries

In [15]:
%reload_ext autoreload
%autoreload 2

from datetime import datetime

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

import lightgbm as lgb
from features import woe

basepath   = os.path.expanduser('../')

# train and validation fold
# TRAIN_PATH = os.path.join(basepath, 'data/processed/application_train_fold.feather')
# TEST_PATH  = os.path.join(basepath, 'data/processed/application_val_fold.feather')

# full training
TRAIN_PATH = os.path.join(basepath, 'data/processed/application_train.feather')
TEST_PATH  = os.path.join(basepath, 'data/processed/application_test.feather')

# MODEL PRESET
MODEL_PRESET   = 'M18'

# DATASET PREFIX
# DATASET_PREFIX = 'tr'
DATASET_PREFIX  = 'train'

## Load Processed Data

In [16]:
%%time

tr = pd.read_feather(f'{TRAIN_PATH}')
te = pd.read_feather(f'{TEST_PATH}')

# Application data from previous loans
bureau       = pd.read_feather(os.path.join(basepath, 'data/processed/bureau.feather'))
bureau_bal   = pd.read_feather(os.path.join(basepath, 'data/processed/bureau_balance.feather'))
prev_app     = pd.read_pickle(os.path.join(basepath, 'data/processed/prev_app.pkl'))
pos_cash     = pd.read_pickle(os.path.join(basepath, 'data/processed/pos_cash.pkl'))
credit_bal   = pd.read_pickle(os.path.join(basepath, 'data/processed/credit_card_balance.pkl'))
installments = pd.read_pickle(os.path.join(basepath, 'data/processed/installments_payments.pkl'))

CPU times: user 558 ms, sys: 1.65 s, total: 2.21 s
Wall time: 11.8 s


In [17]:
# concat training and test set
data   = pd.concat((tr, te))
ntrain = len(tr) 

del tr, te
gc.collect();

## Feature Engineering

In [18]:
%%time

# deviation in three external scores
data.loc[:, 'EXT_SOURCE_DEV']  = data.loc[:, ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(np.std, axis=1)

# sum of external scores
data.loc[:, 'EXT_SOURCE_SUM'] = data['EXT_SOURCE_1'].fillna(0) + data['EXT_SOURCE_2'].fillna(0) + data['EXT_SOURCE_3'].fillna(0)

# mean of external scores
data.loc[:, 'MEAN_EXTERNAL_SCORE'] = (data['EXT_SOURCE_1'].fillna(0) + data['EXT_SOURCE_2'].fillna(0) + data['EXT_SOURCE_3'].fillna(0)) / 3

# feature interactions
data.loc[:, 'EXT_3_1'] = data.loc[:, 'EXT_SOURCE_3'] / data.loc[:, 'EXT_SOURCE_1']
data.loc[:, 'EXT_3_2'] = data.loc[:, 'EXT_SOURCE_3'] / data.loc[:, 'EXT_SOURCE_2']
data.loc[:, 'EXT_2_1'] = data.loc[:, 'EXT_SOURCE_2'] / data.loc[:, 'EXT_SOURCE_1']

# relationship between amount credit and total income
data.loc[:, 'ratio_credit_income'] = data.loc[:, 'AMT_CREDIT'].map(np.log1p) / data.loc[:, 'AMT_INCOME_TOTAL'].map(np.log1p)

# relationship between annual amount to be paid and income
data.loc[:, 'ratio_annuity_income'] = data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'AMT_INCOME_TOTAL']

# relationship between amount annuity and age
data.loc[:, 'ratio_annuity_age'] = data.loc[:, 'AMT_ANNUITY'] / (-data.loc[:, 'DAYS_BIRTH'] / 365)

# number of missing values in an application
data.loc[:, 'num_missing_values'] = data.loc[:, data.columns.drop('TARGET')].isnull().sum(axis=1).values

# feature interaction between age and days employed
data.loc[:, 'age_plus_employed']  = data.loc[:, 'DAYS_BIRTH'] + data.loc[:, 'DAYS_EMPLOYED']
data.loc[:, 'ratio_age_employed'] = (data.DAYS_EMPLOYED) / (data.DAYS_BIRTH)

# ratio of value of goods against which loan is given to total income
data.loc[:, 'ratio_goods_income'] = data.loc[:, 'AMT_GOODS_PRICE'] / data.loc[:, 'AMT_INCOME_TOTAL']

# feature interaction between value of goods against which loan is given to annual loan amount to be paid
data.loc[:, 'ratio_goods_annuity'] = data.loc[:, 'AMT_GOODS_PRICE'] / data.loc[:, 'AMT_ANNUITY']
data.loc[:, 'mult_goods_annuity']  = data.loc[:, 'AMT_GOODS_PRICE'] * data.loc[:, 'AMT_ANNUITY']

# feature interaction value of goods and amount credit
data.loc[:, 'ratio_goods_credit'] = (data.loc[:, 'AMT_GOODS_PRICE'] / data.loc[:, 'AMT_CREDIT']).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'mult_goods_credit']  = (data.loc[:, 'AMT_GOODS_PRICE'] * data.loc[:, 'AMT_CREDIT']).replace([np.inf, -np.inf], np.nan)

# feature interaction between annuity and amount credit
data.loc[:, 'ratio_annuity_credit'] = data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'AMT_CREDIT'].replace([np.inf, -np.inf], np.nan)

# feature interaction between amount credit and age
data.loc[:, 'ratio_credit_age'] = data.AMT_CREDIT / (-data.DAYS_BIRTH / 365)

# feature interaction between amount credit and days before application id was changed
data.loc[:, 'ratio_credit_id_change'] = (data.AMT_CREDIT / -data.DAYS_ID_PUBLISH).replace([np.inf, -np.inf], np.nan)

# feature interaction between days id publish and age
data.loc[:, 'ratio_id_change_age'] = (data.DAYS_ID_PUBLISH / (-data.DAYS_BIRTH / 365))

# ratio of annuity and external score
data.loc[:, 'ratio_annuity_score_1'] = (data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'EXT_SOURCE_1']).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'ratio_annuity_score_2'] = (data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'EXT_SOURCE_2']).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'ratio_annuity_score_3'] = (data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'EXT_SOURCE_3']).replace([np.inf, -np.inf], np.nan)

# ratio of annuity, credit multiplied by external scores
data.loc[:, 'ratio_credit_annuity_score_1'] = ((data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'AMT_CREDIT']) * data.loc[:, 'EXT_SOURCE_1']).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'ratio_credit_annuity_score_2'] = ((data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'AMT_CREDIT']) * data.loc[:, 'EXT_SOURCE_2']).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'ratio_credit_annuity_score_3'] = ((data.loc[:, 'AMT_ANNUITY'] / data.loc[:, 'AMT_CREDIT']) * data.loc[:, 'EXT_SOURCE_3']).replace([np.inf, -np.inf], np.nan)


# ratio of owner's car age with his age
data.loc[:, 'ratio_car_person_age'] = (data.OWN_CAR_AGE / -data.DAYS_BIRTH)

##########################################################################################################
#                            BUREAU                                                                      #
##########################################################################################################

# previous loans
prev_loans = bureau

# number of previous loans for a particular user
prev_num_loans = prev_loans.groupby('SK_ID_CURR').size()

# number of previous active credits
num_active_credits = prev_loans.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].sum()

# mean days credit
mean_days_credit   = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT'].mean()

# median days credit
median_days_credit = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT'].median()

# std days credit
std_days_credit = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT'].std()

# mean number of days overdue on any previous credit
mean_days_overdue = prev_loans.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].mean()

# mean number of days of CB credit at the time of application
mean_days_credit_end = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT_ENDDATE'].mean()

# mean of maximum amount overdue on any credit line
mean_max_amt_overdue = prev_loans.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].mean()

# mean of total amount overdue on any credit line
mean_total_amt_overdue = prev_loans.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].mean()

# sum of num times credit was prolonged
sum_num_times_prolonged = prev_loans.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum()

# median of credit amount of different credit bureau loans
total_credit_amount = prev_loans.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].sum()
mean_credit_amount  = prev_loans.groupby('SK_ID_CURR')['AMT_CREDIT_SUM'].mean()

# number of different types of credit taken from CREDIT BUREAU
num_diff_credits = prev_loans.groupby('SK_ID_CURR')['CREDIT_TYPE'].nunique()

# mean number of days of last credit update
mean_days_credit_update = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT_UPDATE'].mean()    
    
# summary of amount of annuity of credit bureau loans
mean_cb_credit_annuity = prev_loans.groupby('SK_ID_CURR')['AMT_ANNUITY'].mean()
std_cb_credit_annuity  = prev_loans.groupby('SK_ID_CURR')['AMT_ANNUITY'].std()

# latest application reported to Home Credit
latest_credit = prev_loans.groupby('SK_ID_CURR')['DAYS_CREDIT'].max()
data.loc[:, 'latest_credit'] = data.SK_ID_CURR.map(latest_credit)

# day before current application date
credit_duration = (prev_loans.DAYS_CREDIT_ENDDATE - prev_loans.DAYS_CREDIT).map(np.abs).groupby(prev_loans.SK_ID_CURR).mean()
data.loc[:, 'credit_duration'] = data.SK_ID_CURR.map(credit_duration).replace([np.inf, -np.inf], np.nan)

# deviation in difference between remaining duration of credit and how long before we applied for this credit
diff_prev_curr_credit = prev_loans.DAYS_CREDIT_ENDDATE.fillna(0) - prev_loans.DAYS_CREDIT.fillna(0)
diff_prev_curr_credit = diff_prev_curr_credit.groupby(prev_loans.SK_ID_CURR).std()
data.loc[:, 'std_diff_prev_curr_credit'] = data.SK_ID_CURR.map(diff_prev_curr_credit)


# mean of difference between remaining duration of credit and how long before we applied for this credit
diff_prev_curr_credit = prev_loans.DAYS_CREDIT_ENDDATE - prev_loans.DAYS_CREDIT
diff_prev_curr_credit = diff_prev_curr_credit.groupby(prev_loans.SK_ID_CURR).mean()
data.loc[:, 'mean_diff_prev_curr_credit'] = data.SK_ID_CURR.map(diff_prev_curr_credit)

# mean of difference between days since cb credit ended and how long before we applied for current credit
diff_prev_curr_credit = prev_loans.DAYS_ENDDATE_FACT - prev_loans.DAYS_CREDIT
diff_prev_curr_credit = diff_prev_curr_credit.groupby(prev_loans.SK_ID_CURR).mean()
data.loc[:, 'mean_diff_ended_curr_credit'] = data.SK_ID_CURR.map(diff_prev_curr_credit)

# mean of difference between days last credit ended and remaining duration of credit
diff_prev_curr_credit = prev_loans.DAYS_ENDDATE_FACT - prev_loans.DAYS_CREDIT_ENDDATE
diff_prev_curr_credit = diff_prev_curr_credit.groupby(prev_loans.SK_ID_CURR).mean()
data.loc[:, 'mean_diff_prev_remaining_credit'] = data.SK_ID_CURR.map(diff_prev_curr_credit)

# mean of ratio of two differences
diff1 = prev_loans.DAYS_ENDDATE_FACT - prev_loans.DAYS_CREDIT
diff2 = prev_loans.DAYS_CREDIT_ENDDATE - prev_loans.DAYS_CREDIT
diff  = (diff1 / diff2).replace([np.inf, -np.inf], np.nan)
diff  = diff.groupby(prev_loans.SK_ID_CURR).mean()

data.loc[:, 'ratio_two_diff'] = data.SK_ID_CURR.map(diff)

# data.loc[:, 'ratio_total_credit_annuity'] = (data.loc[:, 'AMT_CREDIT'] + active_total_credit) / (data.loc[:, 'AMT_ANNUITY'])

# merge back with original dataframe
data.loc[:, 'num_prev_loans']           = data.SK_ID_CURR.map(prev_num_loans).fillna(0).values
data.loc[:, 'num_prev_active_credits']  = data.SK_ID_CURR.map(num_active_credits).fillna(0).values
data.loc[:, 'mean_days_credit']         = data.SK_ID_CURR.map(mean_days_credit).fillna(0).values
data.loc[:, 'median_days_credit']       = data.SK_ID_CURR.map(median_days_credit).fillna(0).values
data.loc[:, 'std_days_credit']          = data.SK_ID_CURR.map(std_days_credit).fillna(0).values
data.loc[:, 'mean_credit_days_overdue'] = data.SK_ID_CURR.map(mean_days_overdue).fillna(0).values
data.loc[:, 'mean_days_credit_end']     = data.SK_ID_CURR.map(mean_days_credit_end).fillna(0).values
data.loc[:, 'mean_max_amt_overdue']     = data.SK_ID_CURR.map(mean_max_amt_overdue).fillna(0).values
data.loc[:, 'mean_total_amt_overdue']   = data.SK_ID_CURR.map(mean_total_amt_overdue).values
data.loc[:, 'sum_num_times_prolonged']  = data.SK_ID_CURR.map(sum_num_times_prolonged).fillna(0).astype(np.int8).values
data.loc[:, 'total_credit_amount']      = data.SK_ID_CURR.map(total_credit_amount).fillna(0).values
data.loc[:, 'mean_credit_amount']       = data.SK_ID_CURR.map(mean_credit_amount).fillna(0).values
# data.loc[:, 'num_overdue_credits']      = data.SK_ID_CURR.map(num_overdue_credits).fillna(0).astype(np.int8)
data.loc[:, 'mean_cb_credit_annuity']   = data.SK_ID_CURR.map(mean_cb_credit_annuity).fillna(0).values
data.loc[:, 'std_cb_credit_annuity']    = data.SK_ID_CURR.map(std_cb_credit_annuity).fillna(0).values
data.loc[:, 'num_diff_credits']         = data.SK_ID_CURR.map(num_diff_credits).fillna(0).values
data.loc[:, 'mean_days_credit_update']  = data.SK_ID_CURR.map(mean_days_credit_update).fillna(0).values

# load ratio credit overdue to credit sum
# ratio_credit_overdue_sum = pd.read_pickle(os.path.join(basepath, 'data/processed/bureau_ratio_overdue_sum_credit.pkl'))
# data.loc[:, 'ratio_cedit_overdue_sum'] = data.SK_ID_CURR.map(ratio_credit_overdue_sum).fillna(-1)

# interaction between credit amount and duration of credit
credit_times_duration = (prev_loans.AMT_CREDIT_SUM.fillna(0) *\
                         (prev_loans.DAYS_CREDIT_ENDDATE - prev_loans.DAYS_CREDIT).map(np.abs))\
                        .replace([np.inf, -np.inf], np.nan)
credit_times_duration = credit_times_duration.groupby(prev_loans.SK_ID_CURR).mean()
data.loc[:, 'credit_times_duration'] = data.SK_ID_CURR.map(credit_times_duration)


#############################################################################################################
#                      FEATURE INTERACTIONS                                                                 #
#############################################################################################################

# feature interaction between credit bureau annuity, current annuity and total income
data.loc[:, 'ratio_cb_goods_annuity'] = (data.AMT_GOODS_PRICE / (data.mean_cb_credit_annuity + data.AMT_ANNUITY)).replace([np.inf, -np.inf], np.nan)

# feature interaction between mean days credit update and last time id was changed by user
data.loc[:, 'ratio_update_id']        = (data.mean_days_credit_update / data.DAYS_ID_PUBLISH).replace([np.inf, -np.inf], np.nan)

# feature interaction between mean credit amount of previous credits with current credit
data.loc[:, 'ratio_curr_prev_credit'] = data.AMT_CREDIT / data.mean_credit_amount

#############################################################################################################
#                           BUREAU and BUREAU BALANCE                                                       #
#############################################################################################################

prev_bal = bureau.loc[:, ['SK_ID_CURR', 'SK_ID_BUREAU']].merge(bureau_bal,
                                                   on='SK_ID_BUREAU',
                                                   how='left'
                                                  )

mean_status                      = prev_bal.groupby('SK_ID_BUREAU')['STATUS'].mean().fillna(-1)
prev_loans.loc[:, 'mean_status'] = prev_loans.SK_ID_BUREAU.map(mean_status).values

mean_status                = prev_loans.groupby('SK_ID_CURR')['mean_status'].mean()
data.loc[:, 'mean_status'] = data.SK_ID_CURR.map(mean_status).values

# previous loans history
credit_history                = prev_bal.groupby('SK_ID_CURR').size().fillna(0)
data.loc[:, 'credit_history'] = data.SK_ID_CURR.map(credit_history).values 

#############################################################################################################
#                          PREVIOUS APPLICATION                                                             #
#############################################################################################################

# number of previous applications
num_prev_apps                = prev_app.groupby('SK_ID_CURR').size()
data.loc[:, 'num_prev_apps'] = data.SK_ID_CURR.map(num_prev_apps).fillna(0).astype(np.int8) 

# mean amount to be paid annually for previous applications
prev_app_mean_annuity        = prev_app.groupby('SK_ID_CURR')['AMT_ANNUITY'].mean()
prev_app_mean_annuity        = data.SK_ID_CURR.map(prev_app_mean_annuity)

# ratio of previous annuity to current annuity
data.loc[:, 'ratio_prev_curr_annuity'] = (prev_app_mean_annuity / data.AMT_ANNUITY).replace([np.inf, -np.inf], np.nan)
data.loc[:, 'diff_prev_curr_annuity']  = (prev_app_mean_annuity - data.AMT_ANNUITY).replace([np.inf, -np.inf], np.nan)


# ratio of down payment amount to application amount sum
if os.path.exists(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_down_payment_to_application.pkl')):
    down_payment_to_application = pd.read_pickle(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_down_payment_to_application.pkl'))
else:    
    down_payment_to_application = prev_app.groupby('SK_ID_CURR').apply(lambda x: (x['AMT_DOWN_PAYMENT'].fillna(0) / x['AMT_APPLICATION']).sum())
    down_payment_to_application.to_pickle(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_down_payment_to_application.pkl'))

data.loc[:, 'down_payment_to_application'] = data.SK_ID_CURR.map(down_payment_to_application)

# mean interest rate on down payments of previous applications
mean_down_payment_rate                = prev_app.groupby('SK_ID_CURR')['RATE_DOWN_PAYMENT'].mean()
data.loc[:, 'mean_down_payment_rate'] = data.SK_ID_CURR.map(mean_down_payment_rate)

# most frequent rejection reason
if os.path.exists(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_most_freq_reject_reason.pkl')):
    most_freq_rejection_reason = pd.read_pickle(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_most_freq_reject_reason.pkl'))
else:
    most_freq_rejection_reason = prev_app.groupby('SK_ID_CURR').apply(lambda x: x.CODE_REJECT_REASON.value_counts().index.values[0])
    most_freq_rejection_reason.to_pickle(os.path.join(basepath, f'data/processed/{DATASET_PREFIX}_most_freq_reject_reason.pkl'))

data.loc[:, 'most_freq_rejection_reason'] = data.SK_ID_CURR.map(most_freq_rejection_reason)

# median amount annuity
median_annuity                = prev_app.groupby('SK_ID_CURR')['AMT_ANNUITY'].median()
data.loc[:, 'median_annuity'] = data.SK_ID_CURR.map(median_annuity)

# mean of past annuity to credit applications
past_annuity_credit = (prev_app.AMT_ANNUITY / prev_app.AMT_CREDIT).replace([np.inf, -np.inf], np.nan)
past_annuity_credit = past_annuity_credit.groupby(prev_app.SK_ID_CURR).mean()
data.loc[:, 'past_annuity_to_credit'] = data.SK_ID_CURR.map(past_annuity_credit)

# difference between current credit and mean applied amount in previous applications
# mean_applied_amount = prev_app.groupby('SK_ID_CURR')['AMT_APPLICATION'].mean()
# mean_applied_amount = data.SK_ID_CURR.map(mean_applied_amount)
# data.loc[:, 'diff_applied_current_credit'] = mean_applied_amount - data.AMT_CREDIT

# difference of down_payment * rate and annuity
diff_dp_annuity = ((prev_app.AMT_DOWN_PAYMENT * prev_app.RATE_DOWN_PAYMENT) - prev_app.AMT_ANNUITY).replace([np.inf, -np.inf])
diff_dp_annuity = diff_dp_annuity.groupby(prev_app.SK_ID_CURR).sum()
data.loc[:, 'diff_dp_annuity'] = data.SK_ID_CURR.map(diff_dp_annuity)

# mean of decision on last application
mean_last_decision = prev_app.groupby('SK_ID_CURR')['DAYS_DECISION'].mean()
data.loc[:, 'mean_last_decision'] = data.SK_ID_CURR.map(mean_last_decision)

# mean of term of previous credit
mean_prev_credit = prev_app.groupby('SK_ID_CURR')['CNT_PAYMENT'].mean()
data.loc[:, 'mean_prev_credit'] = data.SK_ID_CURR.map(mean_prev_credit)

###############################################################################################################
#                                 POS CASH                                                                    #
###############################################################################################################
cnt_installments_future                = pos_cash.groupby('SK_ID_CURR')['CNT_INSTALMENT_FUTURE'].mean()
data.loc[:, 'cnt_installments_future'] = data.SK_ID_CURR.map(cnt_installments_future)

# mean of term of previous credits
mean_term = pos_cash.groupby('SK_ID_CURR')['CNT_INSTALMENT'].mean()
data.loc[:, 'mean_term'] = data.SK_ID_CURR.map(mean_term)


##############################################################################################################
#                                Credit Card Balance                                                         #
##############################################################################################################

# mean of amount balance during previous payments
mean_amt_balance = credit_bal.groupby('SK_ID_CURR')['AMT_BALANCE'].mean()
data.loc[:, 'mean_amt_balance'] = data.SK_ID_CURR.map(mean_amt_balance)

# mean of actual credit limit
mean_credit_limit = credit_bal.groupby('SK_ID_CURR')['AMT_CREDIT_LIMIT_ACTUAL'].mean()
data.loc[:, 'mean_credit_limit'] = data.SK_ID_CURR.map(mean_credit_limit)

# total paid installments on previous credit
total_paid_installments = credit_bal.groupby('SK_ID_CURR')['CNT_INSTALMENT_MATURE_CUM'].sum()
data.loc[:, 'total_paid_installments'] = data.SK_ID_CURR.map(total_paid_installments)

# mean total drawings
mean_total_drawings                = credit_bal.groupby('SK_ID_CURR')['AMT_DRAWINGS_CURRENT'].mean()
data.loc[:, 'mean_total_drawings'] = data.SK_ID_CURR.map(mean_total_drawings)

# sum of diff between balance and credit limit
diff_bal_credit   = credit_bal.AMT_BALANCE - credit_bal.AMT_CREDIT_LIMIT_ACTUAL
diff_bal_credit   = diff_bal_credit.groupby(credit_bal.SK_ID_CURR).sum()

data.loc[:, 'diff_bal_credit'] = data.SK_ID_CURR.map(diff_bal_credit)

# mean of ratio of balance and credit limit
ratio_bal_credit = credit_bal.AMT_BALANCE / credit_bal.AMT_CREDIT_LIMIT_ACTUAL
ratio_bal_credit = ratio_bal_credit.groupby(credit_bal.SK_ID_CURR).mean()

data.loc[:, 'ratio_bal_credit'] = data.SK_ID_CURR.map(ratio_bal_credit)

###############################################################################################################
#                                  Installment Payments                                                       #
###############################################################################################################

# mean installment
mean_installment = installments.groupby('SK_ID_CURR')['AMT_INSTALMENT'].mean()
data.loc[:, 'mean_installment'] = data.SK_ID_CURR.map(mean_installment)

# mean payment against installment
mean_payment = installments.groupby('SK_ID_CURR')['AMT_PAYMENT'].mean()
data.loc[:, 'mean_payment'] = data.SK_ID_CURR.map(mean_payment)



# delete all intermediatory variables
del prev_num_loans, num_active_credits, prev_loans
del mean_days_credit, median_days_credit
del std_days_credit, mean_days_overdue, mean_days_credit_end
del mean_max_amt_overdue, sum_num_times_prolonged
del mean_cb_credit_annuity, std_cb_credit_annuity
del num_diff_credits, mean_days_credit_update
del mean_status, prev_bal, credit_history
del total_credit_amount, mean_credit_amount
del num_prev_apps
del prev_app, down_payment_to_application
del mean_down_payment_rate, most_freq_rejection_reason
del median_annuity, cnt_installments_future
del latest_credit, credit_duration
del mean_total_amt_overdue, credit_times_duration
del past_annuity_credit, mean_last_decision
del credit_bal, mean_amt_balance, mean_credit_limit
del total_paid_installments, mean_total_drawings
del diff_bal_credit, diff_prev_curr_credit
del diff1, diff2, diff

gc.collect();

CPU times: user 48.2 s, sys: 9.31 s, total: 57.5 s
Wall time: 59.2 s


In [19]:
# replace feature values with frequency less 20 with -100
for f in data.select_dtypes(include=['int8']).columns:
    if data[f].nunique() > 10:        
        low_freq_values = data[f].value_counts()
        low_freq_values = low_freq_values[low_freq_values < 20].index.values
        
        if len(low_freq_values) > 0:
            print('Feature: {}'.format(f))
            data.loc[data[f].isin(low_freq_values), f] = -100

Feature: CNT_CHILDREN
Feature: num_prev_apps


In [20]:
(data.isnull().sum() / len(data)).sort_values(ascending=False)

ratio_bal_credit               0.712010
diff_bal_credit                0.709315
mean_total_drawings            0.709315
total_paid_installments        0.709315
mean_credit_limit              0.709315
mean_amt_balance               0.709315
COMMONAREA_MODE                0.697141
COMMONAREA_AVG                 0.697141
COMMONAREA_MEDI                0.697141
NONLIVINGAPARTMENTS_MODE       0.692933
NONLIVINGAPARTMENTS_MEDI       0.692933
NONLIVINGAPARTMENTS_AVG        0.692933
LIVINGAPARTMENTS_MODE          0.682037
LIVINGAPARTMENTS_AVG           0.682037
LIVINGAPARTMENTS_MEDI          0.682037
FLOORSMIN_AVG                  0.676785
FLOORSMIN_MEDI                 0.676785
FLOORSMIN_MODE                 0.676785
YEARS_BUILD_MODE               0.663306
YEARS_BUILD_MEDI               0.663306
YEARS_BUILD_AVG                0.663306
OWN_CAR_AGE                    0.660316
ratio_car_person_age           0.660316
EXT_3_1                        0.625757
LANDAREA_MODE                  0.591835


In [21]:
# unpack to train and test
tr = data.iloc[:ntrain]
te = data.iloc[ntrain:]

del data
gc.collect();

## Modelling

In [9]:
COLS_TO_REMOVE = [
                  'FLAG_DOCUMENT_21',
                  'FLAG_DOCUMENT_4',
                  'FLAG_MOBIL',
                  'FLAG_DOCUMENT_2',
                  'FLAG_DOCUMENT_20',
                  'FLAG_DOCUMENT_9',
                  'FLAG_DOCUMENT_17',
                  'FLAG_DOCUMENT_19',
                  'FLAG_DOCUMENT_5',
                  'FLAG_CONT_MOBILE',
                  'FLAG_DOCUMENT_10',
                  'HOUSETYPE_MODE',
                  'FLAG_DOCUMENT_12',
                  'FLAG_DOCUMENT_7',
                  'FLAG_DOCUMENT_11',
                  'AMT_REQ_CREDIT_BUREAU_HOUR',
                  'LIVE_REGION_NOT_WORK_REGION',
                  'AMT_REQ_CREDIT_BUREAU_DAY',
                  'FLAG_DOCUMENT_15',
                  'EMERGENCYSTATE_MODE',
                  'REG_REGION_NOT_LIVE_REGION',
                  'SK_ID_CURR',
                  'FLAG_EMP_PHONE',
                  'TARGET'
                ]

In [22]:
features = [f for f in tr.columns if f not in COLS_TO_REMOVE]

Xtr  = tr.loc[:, features]
ytr  = tr.loc[:, 'TARGET']

Xval = te.loc[:, features]
# yval = te.loc[:, 'TARGET'] # only execute during validation phase

del tr, te
gc.collect();

In [23]:
print('Number of features used in the model are: {}'.format(len(features)))

Number of features used in the model are: 176


### Validation

In [12]:
params = {
    'objective': 'binary',
    'learning_rate': .02,
    'metric': 'auc',
    'min_data_in_leaf': 100,
    'num_leaves': 31,
    'feature_fraction': .7,
    'bagging_fraction': .8,
    'min_child_weight': 1.,
    'nthread': 4
}

ltrain = lgb.Dataset(Xtr, ytr, feature_name=features)
lval   = lgb.Dataset(Xval, yval, feature_name=features)

valid_sets  = [ltrain, lval]
valid_names = ['train', 'val']

num_boost_round       = 5000
early_stopping_rounds = 100

m = lgb.train(params, 
              ltrain, 
              num_boost_round, 
              valid_sets=valid_sets, 
              valid_names=valid_names, 
              early_stopping_rounds=early_stopping_rounds, 
              verbose_eval=20)

Training until validation scores don't improve for 100 rounds.
[20]	train's auc: 0.738124	val's auc: 0.73022
[40]	train's auc: 0.740932	val's auc: 0.731619
[60]	train's auc: 0.743761	val's auc: 0.733481
[80]	train's auc: 0.747844	val's auc: 0.736643
[100]	train's auc: 0.752019	val's auc: 0.740838
[120]	train's auc: 0.756095	val's auc: 0.744402
[140]	train's auc: 0.760694	val's auc: 0.748039
[160]	train's auc: 0.765699	val's auc: 0.752066
[180]	train's auc: 0.770673	val's auc: 0.755963
[200]	train's auc: 0.775401	val's auc: 0.759713
[220]	train's auc: 0.779888	val's auc: 0.762968
[240]	train's auc: 0.784022	val's auc: 0.765779
[260]	train's auc: 0.787622	val's auc: 0.767955
[280]	train's auc: 0.790912	val's auc: 0.769732
[300]	train's auc: 0.79401	val's auc: 0.771195
[320]	train's auc: 0.796887	val's auc: 0.772375
[340]	train's auc: 0.799683	val's auc: 0.773507
[360]	train's auc: 0.802352	val's auc: 0.774483
[380]	train's auc: 0.804927	val's auc: 0.775259
[400]	train's auc: 0.807389	val

`[1232]	train's auc: 0.878538	val's auc: 0.781966`

### Feature Importance

In [13]:
# feature importance df
feat_imp = pd.DataFrame({'features': features,
                         'imp': m.feature_importance()
                        })

feat_imp.sort_values(by='imp', ascending=False)

,features,imp
115,ratio_annuity_credit,979
99,EXT_SOURCE_SUM,860
34,EXT_SOURCE_2,742
35,EXT_SOURCE_3,682
20,DAYS_BIRTH,673
166,cnt_installments_future,663
175,mean_payment,616
126,latest_credit,595
113,ratio_goods_credit,566
165,mean_prev_credit,558


In [14]:
feat_imp[feat_imp.features == 'mean_diff_prev_remaining_credit']

,features,imp
131,mean_diff_prev_remaining_credit,299


### OOF Predictions

In [26]:
%%time

HOLDOUT_SCORE  = 0.781892

oof_preds = m.predict(Xval)
joblib.dump(oof_preds, os.path.join(basepath, f'data/oof_sub/{MODEL_PRESET}_{HOLDOUT_SCORE}_preds.pkl'))

CPU times: user 10.8 s, sys: 51 ms, total: 10.9 s
Wall time: 3.44 s


### Full Training

In [24]:
%%time

params = {
    'objective': 'binary',
    'learning_rate': (.02 / 1.2),
    'metric': 'auc',
    'min_data_in_leaf': 100,
    'num_leaves': 31,
    'feature_fraction': .7,
    'bagging_fraction': .8,
    'min_child_weight': 1.,
    'nthread': 4
}

BEST_ITERATION  = 1232

num_boost_round = int(BEST_ITERATION * 1.2)
ltrain          = lgb.Dataset(Xtr, ytr, feature_name=features)

m           = lgb.train(params, ltrain, num_boost_round)
final_preds = m.predict(Xval)

CPU times: user 15min 46s, sys: 2.47 s, total: 15min 48s
Wall time: 4min 23s


In [25]:
HOLDOUT_SCORE  = 0.781966

sub_identifier = "%s-%s-%.5f" % (datetime.now().strftime('%Y%m%d-%H%M'), MODEL_PRESET, HOLDOUT_SCORE)

sub           = pd.read_csv('../data/raw/sample_submission.csv.zip')
sub['TARGET'] = final_preds

sub.to_csv(os.path.join(basepath, 'submissions/%s.csv'%(sub_identifier)), index=False)